Заполните данные о необходимых переменных:

In [1]:
Today = '2024-10-31'

print_Debt = True
print_counterclaims = True
excel_tofolder_on_Z = True


import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'


Открываю следующие файлы:

In [2]:
query = f"""SELECT * FROM "RISKACCESS"."sapPositionArrears" WHERE "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') AND "productType" in (130,131,132,133)"""
data_export = export_from_RISKCUSTOM(query)
data_export['index_SQL'] = data_export.index

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [3]:
data_Debt = data_export.loc[data_export['productTypeName'].isin(['Транш по займам', 'Транш по кредитам'])].reset_index(drop=True)

data_Debt = data_Debt.rename(columns={"businessPartnerName": "Counterparty", "companyName": "Entity", "positionCurrency": "Currency", "purchaseValuePositionCurrency": "amoutn_outstanding"}).reset_index(drop=True)
data_Debt.amoutn_outstanding = data_Debt.amoutn_outstanding.abs()

data_Debt['termEnd'] = data_Debt['termEnd'].fillna('_')
data_Debt = data_Debt.pivot_table(index=['Entity', 'relationshipPartner', 'Counterparty', 'accountAssignmentLinkName', 'Currency', 'termEnd'], values=['purchaseValueDisplayCurrency', 'amoutn_outstanding', 'index_SQL'], aggfunc={'purchaseValueDisplayCurrency': sum, 'amoutn_outstanding': sum, 'index_SQL':list}).reset_index()

data_Debt = add_in_currency_column(data_Debt, col_with_CCY='Currency', col_with_VAL='amoutn_outstanding', DATE=Today, CCY_to='USD', date_is_column=False)
data_Debt['amoutn_outstanding_in_USD'] = data_Debt.amoutn_outstanding_in_USD * np.sign(data_Debt.purchaseValueDisplayCurrency)

data_Debt['Entity_code'] = merge_Mapping(data_Debt, col='Entity')
data_Debt['Entity_group'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='ocpSegment')
data_Debt['Counterparty_code'] = merge_Mapping(data_Debt, col='Counterparty')
data_Debt['Counterparty_Group'] = merge_SalesUnits(data_Debt, col='Counterparty_code', merge_col='ocpSegment')

data_Debt = Period(data_Debt, day_for_count=Today, col_with_date='termEnd')

data_Debt['Holding'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='holding')
data_Debt['termEnd'] = data_Debt['termEnd'].fillna('_')

new_columns = ['dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']
data_Debt = data_Debt.reindex(columns=(data_Debt.columns.tolist() + new_columns))
data_Debt['Source'] = 'SAP'

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11300\580765983.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_Debt['termEnd'] = data_Debt['termEnd'].fillna('_')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11300\580765983.py:7: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_Debt = data_Debt.pivot_table(index=['Entity', 'relationshipPartner', 'Counterparty', 'accountAssignmentLinkName', 'Currency', 'termEnd'], values=['purchaseValueDisplayCurrency', 'amoutn_outstanding', 'index_SQL'], aggfunc={'purchaseValueDisplayCurrency': sum, 'amoutn_outstanding': sum, 'index_

Встречные требования по валютам:

In [4]:
# part of the data by holding
holding = ''
for holding in ['SUEK', 'EUROCHEM']:

    data_E_to_C_holding = data_Debt[(data_Debt.Holding == holding) & (data_Debt.Counterparty_Group != "External")]
    startrow = 1
    Currency = ''


    # matrix for each holding
    Entity_group_list = list(data_E_to_C_holding.Entity_group.unique())
    Entity_group_list.sort(reverse=True)
    Counterparty_Group_list = list(data_E_to_C_holding.Counterparty_Group.unique())
    Counterparty_Group_list.sort(reverse=True)
    matrix_list = []
    for Entity_group in Entity_group_list:
        matrix_dict = {}
        matrix_dict['Entity_group'] = Entity_group
        for Counterparty_Group in Counterparty_Group_list:
            matrix_dict[Counterparty_Group] = 0
        matrix_list.append(matrix_dict)
    matrix_df = pd.DataFrame(matrix_list)
    matrix_df.index = matrix_df['Entity_group']
    matrix_df = matrix_df.drop('Entity_group',axis=1)

    ### to excel
    if print_counterclaims == True:
        Output_file = "_".join([str(date.today()), holding, 'SAP_Debt.xlsx'])
        if excel_tofolder_on_Z == True:
            Output_file = Output_path + Output_file
        
        writer = pd.ExcelWriter(Output_file, engine='openpyxl')  
        sheet_name = f'{holding}_CCY_counterclaims'
        pd.DataFrame({'holding':f'{holding}_by_CCY'}, index=[1]).to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    # part of the data by CCY
    for Currency in data_E_to_C_holding.Currency.unique().tolist():
        data_E_to_C_Currency = data_E_to_C_holding[data_E_to_C_holding.Currency == Currency]
        pivot = data_E_to_C_Currency.pivot_table(index=['Entity_group','Counterparty_Group'],values='amoutn_outstanding_in_USD',aggfunc=sum).reset_index()
        
        matrix_df[:] = 0
        ccy_df = matrix_df
        for string in Entity_group_list:
            for col in Counterparty_Group_list:
                CCY_df_part = pivot[(pivot.Entity_group==string)&(pivot.Counterparty_Group==col)]
                try:
                    ccy_df.loc[string, col] = CCY_df_part.amoutn_outstanding_in_USD.values[0]
                except:
                    pass

        ### to excel
        if print_counterclaims == True:
            pd.DataFrame([{'CCY':Currency}]).to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=startrow)
            startrow += 1
            ccy_df.to_excel(writer, sheet_name=sheet_name, index=True, startrow=startrow)
            startrow = startrow + len(ccy_df) + 2
    if print_counterclaims == True:
        writer.close()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11300\4289885755.py:39: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot = data_E_to_C_Currency.pivot_table(index=['Entity_group','Counterparty_Group'],values='amoutn_outstanding_in_USD',aggfunc=sum).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11300\4289885755.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-200760730.2767631' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ccy_df.loc[string, col] = CCY_df_part.amoutn_outstanding_in_USD.values[0]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11300\4289885755.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will rais

In [5]:
Sheet_in_output_file = 'Debt'

if print_Debt == True:
    for group in ['SUEK', 'EUROCHEM']:
        data_Debt_group = data_Debt[data_Debt.Holding == group]
        data_Debt_print = data_Debt_group[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amoutn_outstanding_in_USD', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
        Output_file = "_".join([str(date.today()), group, 'SAP_Debt.xlsx'])
        if excel_tofolder_on_Z == True:
            Output_file = Output_path + Output_file
        new_list(data_Debt_print, Output_file, Sheet_in_output_file, index=False)